In [1]:
import torchvision.models as models
import torch
from torchvision.transforms import v2
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler
import torchvision.datasets as datasets

# import os
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# from PIL import Image
import numpy as np


import random
import numpy as np
from sklearn.metrics import f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.metrics import classification_report


In [2]:
class TrainingDataset(Dataset):
    
    def __init__(self, dataset, augmentation ):
        self.dataset = dataset

    def __len__(self):
        return len(dataset)

    def __getitem__(self, idx):
        image_path, label = self.dataset.samples[idx]  
        image = self.dataset.loader(image_path) 
        image = augmentation(image)

        return image, label

**Augumentation**


In [3]:
augmentation = transforms.Compose([
      
    transforms.Resize((224, 224)),

    transforms.RandomRotation(degrees=(0, 10)), 
    
    transforms.RandomApply([transforms.RandomHorizontalFlip()], p=0.5),

    transforms.RandomApply([transforms.RandomVerticalFlip()], p=0.5),

    transforms.ToTensor()
    
])

In [4]:
class ResNetTransformerClassifier(nn.Module):
    
    def __init__(self, num_classes=4, d_model=2048, num_heads=8, num_layers=2):
        
        super(ResNetTransformerClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  
  
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads,  batch_first=True )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        
        x = self.resnet(x)  
        x = x.unsqueeze(1)  
        
        x = self.transformer(x)  
        x = x.squeeze(1)
        
        return self.fc(x) 

In [5]:
dataset_path = '/kaggle/input/trining-dataset/training'
validationDataset_path = '/kaggle/input/validation/validating'

training_dataset = datasets.ImageFolder(root=dataset_path)
validation_dataset = datasets.ImageFolder(root=validationDataset_path)

**DATA STATISTICS**

In [6]:
len(training_dataset)

33673

In [7]:
len(validation_dataset)

14439

In [8]:
def get_statistics(dataset):
    class_counts = {'A':0, 'P':0, 'N':0, 'E':0}
    for _, label in dataset.samples:
        class_counts[dataset.classes[label]] += 1

    print("the statistics ---")

    print("Total number --")
    
    for key, value in enumerate(class_counts):
        print(value, ' ', class_counts[value])

    print("Now, let go to the percentage :::")

    len_ = len(dataset)

    for key, value in enumerate(class_counts):
        print(class_counts[value]/len_)

In [9]:
get_statistics(training_dataset)

the statistics ---
Total number --
A   1154
P   1162
N   28663
E   2694
Now, let go to the percentage :::
0.03427078074421643
0.03450835981350043
0.8512161078608974
0.08000475158138567


**FROM STATISTICS - UNDERSAMPLING CONCLUSION**

- concluded the undersample percentage of the erosion will be 0.1

In [18]:
target_class = "N"  
percentage = 0.1

class_to_idx = training_dataset.class_to_idx
target_class_idx = class_to_idx[target_class]

class_indices = [
    
    i for i, (_, label) in enumerate(training_dataset.samples) 
    if label == target_class_idx
    
]

num_samples = int(len(class_indices) * percentage)
selected_indices = random.sample(class_indices, num_samples)

**loading the dataset**

**NOW, LET DEFINE THE INDEXES**

In [11]:
# finding first, last indexex, + middle indexes + last indexes

labels = np.array([label for _, label in training_dataset.samples])

In [12]:
def find_class_indices(dataset, class_name):
    """Find first and last index of a class in ImageFolder dataset."""
    class_idx = dataset.class_to_idx[class_name]
    indices = np.where(labels == class_idx)[0]   
    
    if len(indices) == 0:
        return None, None  # Class not found
    
    return indices[0], indices[-1]  # First and last occurrence

In [13]:
first_idx_P, last_idx_P = find_class_indices(training_dataset, 'P')
first_idx_A, last_idx_A = find_class_indices(training_dataset, 'A')
first_idx_E, last_idx_E = find_class_indices(training_dataset, 'E')

In [14]:
last_idx_A , first_idx_E

(1153, 1154)

In [15]:
# CONCUDED EXCLUSIVE INTERVALS
ADD_a = [ x for x in range(first_idx_A, last_idx_A +1 )]
ADD_p =  [ x for x in range(first_idx_P, last_idx_P +1 )]
ADD_e =  [ x for x in range(first_idx_E, last_idx_E +1 )]

In [19]:
final_indexes = ADD_a + ADD_p + ADD_e  + selected_indices 

In [20]:
len(final_indexes)

7876

**DEFINING THE SAMPLER**

In [15]:
train_dataset = TrainingDataset(training_dataset, augmentation)

In [16]:
class UnderSampler(Sampler):
    
    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return iter(random.sample(self.indices, len(self.indices)))  # Returns shuffled indices

    def __len__(self):
        return len(self.indices)


In [47]:
train_loader = DataLoader(train_dataset, batch_size=64, sampler=UnderSampler(final_indexes)
                         
                     )

In [2]:
# for image, label in train_loader:
#     print(label)
    

In [18]:
class ValidationDataset(Dataset):
    
    def __init__(self, dataset, augmentation):
        self.dataset = dataset
        self.augmentation = augmentation
        
    def __len__(self):
        return 7876

    def __getitem__(self, idx):
        
        image_path, label = validation_dataset.samples[idx]
        image =   validation_dataset.loader(image_path) 

        image = self.augmentation(image)

        return image, label

**loading validation dataset**

In [19]:
validation_dataset2 = ValidationDataset(validation_dataset, augmentation)

In [20]:
valid_loader = DataLoader( validation_dataset2, batch_size=128 )

In [48]:
model = ResNetTransformerClassifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [49]:
trainable_params = list(model.transformer.parameters()) + list(model.fc.parameters())

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-1)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

**testing certieria**

In [50]:
criterion = nn.CrossEntropyLoss()

In [51]:

num_epochs = 80

patience = 10
best_metric = -np.inf  

epochs_no_improve = 0

In [52]:
for epoch in range(num_epochs):
    
    model.train()
    
    running_loss = 0.0
    
    for images, labels in train_loader:
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)

        loss = criterion(outputs, labels).to(device)
        
        loss.backward()

  #      print(loss)
        
        optimizer.step()
        
        running_loss += loss.item()

    # calculate the loss

    scheduler.step()

    preds_ = []
    real_ = []
    
    with torch.no_grad():
    
        for images, labels in train_loader:
        
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            preds_.extend(preds.cpu().numpy()) 
            real_.extend(labels.cpu().numpy())

        no_trues= {0:0, 1:0, 2:0, 3:0}
        no_r_t = {0:0, 1:0, 2:0, 3:0}
        
        for p1, t1 in zip(preds_, real_):

            if p1 == t1 :
                no_trues[t1] += 1
                
            no_r_t[t1] += 1
            
    print(no_trues)
    print(no_r_t)

{0: 0, 1: 2672, 2: 28, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 6, 1: 2254, 2: 1666, 3: 43}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2480, 2: 1609, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2485, 2: 1657, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2503, 2: 1631, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2525, 2: 1630, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2516, 2: 1645, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2518, 2: 1646, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2523, 2: 1641, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2521, 2: 1650, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2515, 2: 1648, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2514, 2: 1647, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2518, 2: 1653, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2518, 2: 1650, 3: 0}
{0: 1154, 1: 2694, 2: 2006, 3: 1162}
{0: 0, 1: 2517, 2: 1642, 3: 0}
{0: 1154, 1: 2694,

KeyboardInterrupt: 

In [1]:
# import torch
# import torch.nn as nn
# import torchvision.models as models

# # Load a pretrained ResNet model
# resnet = models.resnet50(pretrained=True)

# # Remove the final fully connected layer
# resnet.fc = nn.Identity()

# # Forward pass through modified ResNet
# x = torch.randn(1, 3, 224, 224)  # Example input
# features = resnet(x)

# print(features.shape)  # Output feature shape instead of class scores
